In [2]:
import numpy as np
from keras import Input
from keras.optimizers import Adam
from keras.utils import multi_gpu_model
from config import *
from model import unet
from librosa.util import find_files

## Load preprocessed magnitude data
Preprocessed data are stored as npz dictionaries in `./numpy`. We will load only the mix track, as well as the target track(inst or vocal) we're interested in.

In [3]:
def load_npz(target=None, first=None):
    npz_files = find_files('numpy', ext="npz")[:first]
    for file in npz_files:
        npz = np.load(file)
        assert(npz["mix"].shape == npz[target].shape)
        yield npz['mix'], npz[target]
mix_mag, target_mag = zip(*load_npz(target='vocal', first=-1))

## Compile UNet model

In [4]:
model = unet()
raw_model = model
model.compile(optimizer=Adam(1e-4), loss='mean_absolute_error')

## Sample from audio data and train model
Small patches are extracted from the audio data for training.

In below training strategy, different inputs are generated in each epoch.

In [6]:
EPOCH = 20
BATCH = 16
SAMPLING_STRIDE = 10

def sampling(mix_mag, target_mag):
    X, y = [], []
    for mix, target in zip(mix_mag, target_mag):
        starts = np.random.randint(0, mix.shape[1] - PATCH_SIZE, (mix.shape[1] - PATCH_SIZE) // SAMPLING_STRIDE)
        for start in starts:
            end = start + PATCH_SIZE
            X.append(mix[1:, start:end, np.newaxis])
            y.append(target[1:, start:end, np.newaxis])
    return np.asarray(X, dtype=np.float32), np.asarray(y, dtype=np.float32)

for e in range(EPOCH):
    X, y = sampling(mix_mag, target_mag)
    model.fit(X, y, batch_size=BATCH, verbose=1, validation_split=0.05)
    model.save('vocal_{:0>2d}.h5'.format(e+1), overwrite=True)

Train on 11453 samples, validate on 603 samples
Epoch 1/1
11453/11453 [==============================] - 128s 11ms/step - loss: 0.0118 - val_loss: 0.0090
Train on 11453 samples, validate on 603 samples
Epoch 1/1
11453/11453 [==============================] - 129s 11ms/step - loss: 0.0066 - val_loss: 0.0055
Train on 11453 samples, validate on 603 samples
Epoch 1/1
11453/11453 [==============================] - 130s 11ms/step - loss: 0.0053 - val_loss: 0.0050
Train on 11453 samples, validate on 603 samples
Epoch 1/1
11453/11453 [==============================] - 131s 11ms/step - loss: 0.0048 - val_loss: 0.0050
Train on 11453 samples, validate on 603 samples
Epoch 1/1
11453/11453 [==============================] - 131s 11ms/step - loss: 0.0047 - val_loss: 0.0044
Train on 11453 samples, validate on 603 samples
Epoch 1/1
11453/11453 [==============================] - 131s 11ms/step - loss: 0.0044 - val_loss: 0.0042
Train on 11453 samples, validate on 603 samples
Epoch 1/1
11453/11453 [=====